# SHAP feature selection #
## Code to select feature with combination of LightGBM and SHAP ##

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 5
importance_type = "split"

life_history = ["lifespan", "mass_kg", "mtGC", "metabolic_rate", "temperature", "gestation_days"]
debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

#lgb_params["importance_type"] = importance_type

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.models import Metrics
from yspecies.selection import ShapSelector
from yspecies.results import FeatureSummary

In [5]:
import optuna
from optuna import Study, Trial
from optuna import multi_objective
from optuna.multi_objective import trial
from optuna.multi_objective.study import MultiObjectiveStudy
from yspecies.tuning import MultiObjectiveResults

In [6]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [7]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

## Setting up Features to select ##

In [8]:
default_selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type =  importance_type,
    feature_perturbation = "tree_path_dependent"
)
default_selection

Samples metadata,Species metadata,Genes,Predict label,not_validated species
"['tissue', 'species']",[],all,lifespan,[]


In [9]:
loader = DataLoader(locations, default_selection)
selections = loader.load_life_history()
selections["lifespan"][0]

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 12323)","(12323, 37)",38,408,"(12323, 2)","(38, 18)"


### Loading Hyperoptimization studies ###

In [10]:
def load_study(trait: str):
    url = f'sqlite:///' +str((locations.interim.optimization / (trait+".sqlite")).absolute())
    print('loading (if exists) study from '+url)
    storage = optuna.storages.RDBStorage(
        url=url
        #engine_kwargs={'check_same_thread': False}
    )
    return optuna.multi_objective.study.create_study(directions=['maximize','minimize','maximize'], storage = storage, study_name = f"{trait}_r2_huber_kendall", load_if_exists = True)

## Setting up SHAP selection pipeline ##

### Deciding on selection parameters (which fields to include, exclude, predict)  ###

In [11]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2,   42)


In [12]:
def make_pipeline(trait: str):
    study = load_study(trait)
    if len(study.get_pareto_front_trials())>0 :
        metrics, params = MultiObjectiveResults.from_study(study).best_metrics_params_r2()
    else:
        params =  lgb_params = {"bagging_fraction": 0.9522534844058304,
                  "boosting_type": "dart",
                  "objective": "regression",
                  "feature_fraction": 0.42236910941558053,
                  "lambda_l1": 0.020847266580277746,
                  "lambda_l2": 2.8448564854773326,
                  "learning_rate": 0.11484015430016059,
                  "max_depth": 3,
                  "max_leaves": 35,
                  "min_data_in_leaf": 9,
                  "num_iterations": 250,
                  "metrics": ["l1", "l2", "huber"]
                 }
    partition_shap_pipe = Pipeline([
    ("partitioner", DataPartitioner()),
    ('prepare_for_selection', TupleWith(params)),
    ("shap_computation", ShapSelector())
    ]
    )
    repeated_cv =  Repeat(partition_shap_pipe, repeats, lambda x,i: (x[0], replace(x[1], seed = i)))
    return Pipeline(
        [
        ('extractor', DataExtractor()),
        ('prepare_for_partitioning', TupleWith(partition_params)), # to extract the data required for ML from the dataset
        ("partition_shap", repeated_cv),
        ("summarize", Collect(fold=lambda results: FeatureSummary(results)))
        ]
        )

In [13]:
pipelines = {trait:make_pipeline(trait) for trait in life_history}
pipelines.keys()

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/lifespan.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-30 15:15:22,045] Using an existing study with name 'lifespan_r2_huber_kendall' instead of creating a new one.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/mass_kg.sqlite


[I 2020-08-30 15:15:22,889] Using an existing study with name 'mass_kg_r2_huber_kendall' instead of creating a new one.


loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/mtGC.sqlite


...

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/temperature.sqlite


[I 2020-08-30 15:15:24,485] Using an existing study with name 'temperature_r2_huber_kendall' instead of creating a new one.


loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/gestation_days.sqlite


[I 2020-08-30 15:15:24,737] Using an existing study with name 'gestation_days_r2_huber_kendall' instead of creating a new one.


dict_keys(['lifespan', 'mass_kg', 'mtGC', 'metabolic_rate', 'temperature', 'gestation_days'])

# First stage selection (shap ) #

In [16]:
stage_one = {}

### Lifespan ###

In [19]:
def run_stage_one(trait: str):
    result = pipelines[trait].fit_transform(selections[trait])
    stage_one[trait] = result #ugly mutation of stage_one dicionary
    return result

In [20]:
stage_one_lifespan = run_stage_one("lifespan")
stage_one_lifespan.selected

2020-08-30 15:20:37.106 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 0 =====
2020-08-30 15:20:37.107 | INFO     | yspecies.selection:fit:79 - PARAMETERS:
{'bagging_fraction': 0.9223944053685549, 'boosting_type': 'gbdt', 'drop_rate': 0.29847846764282016, 'feature_fraction': 0.30501714460224083, 'lambda_l1': 0.21894440516534, 'lambda_l2': 1.3848191603958375, 'learning_rate': 0.08210966706622723, 'max_depth': 3, 'max_leaves': 17, 'min_data_in_leaf': 8, 'objective': 'regression', 'metrics': ['l1', 'l2', 'huber']}
2020-08-30 15:20:37.136 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Gorilla_gorilla', 'Pan_paniscus']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[158]	valid_0's l1: 8.56432	valid_0's l2: 189.457	valid_0's huber: 7.35476


2020-08-30 15:20:40.328 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Heterocephalus_glaber', 'Capra_hircus']


Training until validation scores don't improve for 10 rounds
[250]	valid_0's l1: 3.93228	valid_0's l2: 62.8388	valid_0's huber: 3.21679
Did not meet early stopping. Best iteration is:
[250]	valid_0's l1: 3.93228	valid_0's l2: 62.8388	valid_0's huber: 3.21679


2020-08-30 15:20:45.353 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Bos_taurus', 'Otolemur_garnettii']


...

2020-08-30 15:22:53.686 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 2 | VALIDATION_SPECIES: ['Heterocephalus_glaber', 'Cavia_aperea']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[97]	valid_0's l1: 5.05256	valid_0's l2: 59.1748	valid_0's huber: 4.2051


2020-08-30 15:22:57.589 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 3 | VALIDATION_SPECIES: ['Pan_paniscus', 'Gorilla_gorilla']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[123]	valid_0's l1: 6.38498	valid_0's l2: 125.938	valid_0's huber: 5.39846


,symbol,repeats,mean_shap,mean_kendall_tau,shap_0,kendall_tau_0,shap_1,kendall_tau_1,shap_2,kendall_tau_2,shap_3,kendall_tau_3,shap_4,kendall_tau_4
ENSG00000204498,NFKBIL1,5.000,7.600,-0.106,7.000,-0.156,7.750,-0.036,8.000,-0.124,7.500,-0.072,7.750,-0.140
ENSG00000010219,DYRK4,5.000,7.550,0.277,8.000,0.306,8.750,0.283,7.500,0.262,8.500,0.272,5.000,0.261
ENSG00000185880,TRIM69,5.000,4.650,0.145,4.250,0.102,4.000,0.203,4.000,0.136,3.500,0.090,7.500,0.193
ENSG00000105672,ETV2,5.000,3.550,0.728,2.750,0.796,3.000,0.723,2.750,0.734,4.250,0.680,5.000,0.708
ENSG00000167515,TRAPPC2L,4.000,5.750,-0.210,nan,nan,4.750,-0.297,4.250,-0.180,7.250,-0.180,6.750,-0.183
ENSG00000132436,FIGNL1,4.000,3.938,0.506,4.000,0.468,nan,nan,3.250,0.521,3.750,0.525,4.750,0.512
ENSG00000165501,LRR1,4.000,2.125,0.694,2.250,0.669,nan,nan,2.000,0.742,1.500,0.652,2.750,0.712
ENSG00000188747,NOXA1,4.000,2.062,0.696,1.250,0.679,nan,nan,2.250,0.706,2.250,0.684,2.500,0.716
ENSG00000066923,STAG3,3.000,6.333,0.449,nan,nan,5.000,0.380,8.500,0.453,nan,nan,5.500,0.514
ENSG00000171121,KCNMB3,3.000,4.417,0.668,5.000,0.704,4.000,0.654,nan,nan,4.250,0.646,nan,nan


### Mass_kg ###

In [21]:
stage_one_mass = run_stage_one("mass_kg")
stage_one_mass.selected

2020-08-30 15:23:17.659 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 0 =====
2020-08-30 15:23:17.660 | INFO     | yspecies.selection:fit:79 - PARAMETERS:
{'bagging_fraction': 0.49885668399953875, 'boosting_type': 'dart', 'drop_rate': 0.2372958868935867, 'feature_fraction': 0.49141007534802034, 'lambda_l1': 0.656412473127467, 'lambda_l2': 1.9535470163293591, 'learning_rate': 0.08702343727058373, 'max_depth': 3, 'max_leaves': 21, 'min_data_in_leaf': 8, 'objective': 'regression', 'metrics': ['l1', 'l2', 'huber']}
2020-08-30 15:23:17.676 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Monodelphis_domestica', 'Aotus_nancymaae']
Early stopping is not available in dart mode


[250]	valid_0's l1: 13.6373	valid_0's l2: 469.528	valid_0's huber: 11.8875


2020-08-30 15:23:22.375 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Sus_scrofa', 'Oryctolagus_cuniculus']


[250]	valid_0's l1: 32.1894	valid_0's l2: 2498.65	valid_0's huber: 28.5926


2020-08-30 15:23:27.002 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Mesocricetus_auratus', 'Cavia_porcellus']


...

2020-08-30 15:25:50.473 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 2 | VALIDATION_SPECIES: ['Homo_sapiens', 'Rattus_norvegicus']


[250]	valid_0's l1: 17.6793	valid_0's l2: 921.116	valid_0's huber: 15.5183


2020-08-30 15:25:54.151 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 3 | VALIDATION_SPECIES: ['Phascolarctos_cinereus', 'Ictidomys_tridecemlineatus']


[250]	valid_0's l1: 15.7326	valid_0's l2: 461.445	valid_0's huber: 13.7654


,symbol,repeats,mean_shap,mean_kendall_tau,shap_0,kendall_tau_0,shap_1,kendall_tau_1,shap_2,kendall_tau_2,shap_3,kendall_tau_3,shap_4,kendall_tau_4
ENSG00000054277,OPN3,4.000,48.312,0.124,29.250,-0.048,nan,nan,66.000,0.395,29.500,0.025,68.500,0.123
ENSG00000144214,LYG1,4.000,11.688,0.370,9.500,0.434,17.000,0.400,12.500,0.293,nan,nan,7.750,0.351
ENSG00000164304,CAGE1,3.000,14.167,-0.272,20.000,-0.284,3.750,-0.343,nan,nan,18.750,-0.188,nan,nan
ENSG00000270647,TAF15,3.000,12.417,0.008,nan,nan,12.500,-0.032,10.250,-0.092,nan,nan,14.500,0.148
ENSG00000124172,ATP5F1E,3.000,11.833,-0.314,nan,nan,8.500,-0.255,3.000,-0.251,24.000,-0.437,nan,nan
ENSG00000126062,TMEM115,3.000,9.000,-0.032,6.000,0.053,9.500,-0.073,nan,nan,nan,nan,11.500,-0.075
ENSG00000066923,STAG3,2.000,35.000,0.173,37.000,0.219,nan,nan,nan,nan,33.000,0.127,nan,nan
ENSG00000127952,STYXL1,2.000,19.375,0.330,20.000,0.195,nan,nan,nan,nan,18.750,0.465,nan,nan
ENSG00000010219,DYRK4,2.000,16.625,0.240,18.750,0.182,nan,nan,nan,nan,14.500,0.298,nan,nan
ENSG00000107020,PLGRKT,2.000,11.500,-0.011,10.000,-0.063,13.000,0.041,nan,nan,nan,nan,nan,nan


### MtGC ###

In [22]:
stage_one_mtGC = run_stage_one("mtGC")
stage_one_mtGC.selected

2020-08-30 15:26:13.439 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 0 =====
2020-08-30 15:26:13.440 | INFO     | yspecies.selection:fit:79 - PARAMETERS:
{'bagging_fraction': 0.4882827581871194, 'boosting_type': 'gbdt', 'drop_rate': 0.16290376621758335, 'feature_fraction': 0.3059934876406196, 'lambda_l1': 0.017007142345511594, 'lambda_l2': 0.568546479848152, 'learning_rate': 0.08231879540454069, 'max_depth': 7, 'max_leaves': 24, 'min_data_in_leaf': 8, 'objective': 'regression', 'metrics': ['l1', 'l2', 'huber']}
2020-08-30 15:26:13.459 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Callithrix_jacchus', 'Sus_scrofa']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[104]	valid_0's l1: 0.434639	valid_0's l2: 0.398746	valid_0's huber: 0.168501


2020-08-30 15:26:19.259 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Rattus_norvegicus', 'Gorilla_gorilla']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[38]	valid_0's l1: 0.840033	valid_0's l2: 1.03454	valid_0's huber: 0.440867


2020-08-30 15:26:21.446 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Macaca_mulatta', 'Equus_caballus']


...

2020-08-30 15:28:16.548 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 2 | VALIDATION_SPECIES: ['Pan_troglodytes', 'Heterocephalus_glaber']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[145]	valid_0's l1: 0.737914	valid_0's l2: 1.36587	valid_0's huber: 0.439515


2020-08-30 15:28:22.673 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 3 | VALIDATION_SPECIES: ['Capra_hircus', 'Tupaia_belangeri']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[127]	valid_0's l1: 0.69111	valid_0's l2: 1.32116	valid_0's huber: 0.400158


,symbol,repeats,mean_shap,mean_kendall_tau,shap_0,kendall_tau_0,shap_1,kendall_tau_1,shap_2,kendall_tau_2,shap_3,kendall_tau_3,shap_4,kendall_tau_4
ENSG00000171121,KCNMB3,5.000,10.300,0.639,12.000,0.661,7.750,0.612,14.000,0.631,7.500,0.630,10.250,0.662
ENSG00000122034,GTF3A,5.000,9.850,0.622,10.000,0.610,11.000,0.626,9.750,0.681,10.250,0.669,8.250,0.524
ENSG00000144451,SPAG16,5.000,6.050,0.348,5.750,0.322,5.750,0.310,5.750,0.295,3.500,0.353,9.500,0.460
ENSG00000196419,XRCC6,4.000,11.562,0.656,10.250,0.686,14.250,0.614,7.500,0.697,nan,nan,14.250,0.626
ENSG00000188747,NOXA1,4.000,6.250,0.648,5.250,0.715,11.750,0.561,6.500,0.656,nan,nan,1.500,0.661
ENSG00000167515,TRAPPC2L,4.000,4.562,-0.556,4.250,-0.534,5.250,-0.546,2.500,-0.585,nan,nan,6.250,-0.561
ENSG00000164304,CAGE1,4.000,3.938,-0.731,5.750,-0.773,3.750,-0.700,2.500,-0.702,nan,nan,3.750,-0.747
ENSG00000156384,SFR1,4.000,3.312,-0.662,4.000,-0.686,4.500,-0.646,nan,nan,2.250,-0.648,2.500,-0.669
ENSG00000048140,TSPAN17,4.000,2.062,-0.401,nan,nan,1.750,-0.340,1.750,-0.433,1.250,-0.525,3.500,-0.307
ENSG00000006282,SPATA20,3.000,8.667,0.677,nan,nan,6.000,0.655,9.750,0.684,nan,nan,10.250,0.693


### Metabolism ###

In [26]:
stage_one_metabolic_rate = run_stage_one("metabolic_rate")
stage_one_metabolic_rate.selected

2020-08-30 15:38:10.081 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 0 =====
2020-08-30 15:38:10.082 | INFO     | yspecies.selection:fit:79 - PARAMETERS:
{'bagging_fraction': 0.7949279232062532, 'boosting_type': 'dart', 'drop_rate': 0.19407161947050122, 'feature_fraction': 0.8486129005166576, 'lambda_l1': 2.9464767054398204, 'lambda_l2': 0.817423041659147, 'learning_rate': 0.08579903520990408, 'max_depth': 7, 'max_leaves': 15, 'min_data_in_leaf': 8, 'objective': 'regression', 'metrics': ['l1', 'l2', 'huber']}
2020-08-30 15:38:10.098 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Monodelphis_domestica', 'Cavia_porcellus']
Early stopping is not available in dart mode


[250]	valid_0's l1: 13.689	valid_0's l2: 323.66	valid_0's huber: 11.9461


2020-08-30 15:38:16.472 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Sus_scrofa', 'Phascolarctos_cinereus']


[250]	valid_0's l1: 24.1289	valid_0's l2: 1198.32	valid_0's huber: 21.3451


2020-08-30 15:38:26.632 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Meriones_unguiculatus', 'Rattus_norvegicus']


...

2020-08-30 15:42:16.122 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 2 | VALIDATION_SPECIES: ['Ictidomys_tridecemlineatus', 'Oryctolagus_cuniculus']


[250]	valid_0's l1: 16.7229	valid_0's l2: 669.705	valid_0's huber: 14.6837


2020-08-30 15:42:25.870 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 3 | VALIDATION_SPECIES: ['Callithrix_jacchus', 'Monodelphis_domestica']


[250]	valid_0's l1: 18.325	valid_0's l2: 528.165	valid_0's huber: 16.0927


,symbol,repeats,mean_shap,mean_kendall_tau,shap_0,kendall_tau_0,shap_1,kendall_tau_1,shap_2,kendall_tau_2,shap_3,kendall_tau_3,shap_4,kendall_tau_4
ENSG00000054277,OPN3,3.000,51.417,0.129,2.750,0.037,nan,nan,77.000,0.204,nan,nan,74.500,0.145
ENSG00000164304,CAGE1,3.000,25.833,-0.248,34.500,-0.353,6.750,-0.267,nan,nan,36.250,-0.126,nan,nan
ENSG00000175806,MSRA,3.000,23.167,-0.076,24.250,-0.022,15.000,-0.004,nan,nan,30.250,-0.204,nan,nan
ENSG00000270647,TAF15,3.000,21.750,-0.019,nan,nan,25.000,0.006,27.000,-0.015,13.250,-0.048,nan,nan
ENSG00000124172,ATP5F1E,3.000,16.667,-0.261,23.500,-0.219,11.250,-0.235,nan,nan,15.250,-0.328,nan,nan
ENSG00000171747,LGALS4,3.000,10.083,-0.601,14.000,-0.650,nan,nan,5.250,-0.648,nan,nan,11.000,-0.504
ENSG00000156467,UQCRB,3.000,3.667,-0.263,nan,nan,4.750,-0.433,2.250,-0.525,4.000,0.168,nan,nan
ENSG00000101546,RBFA,3.000,3.250,0.473,2.750,0.215,3.250,0.719,nan,nan,nan,nan,3.750,0.484
ENSG00000103005,USB1,3.000,2.667,0.106,4.750,0.185,1.000,0.513,nan,nan,nan,nan,2.250,-0.382
ENSG00000010219,DYRK4,2.000,59.250,0.347,nan,nan,nan,nan,113.000,0.182,5.500,0.512,nan,nan


### Temperature ###

In [27]:
stage_one_temperature = run_stage_one("temperature")
stage_one_temperature.selected

2020-08-30 15:42:51.437 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 0 =====
2020-08-30 15:42:51.438 | INFO     | yspecies.selection:fit:79 - PARAMETERS:
{'bagging_fraction': 0.9219040738483888, 'boosting_type': 'gbdt', 'drop_rate': 0.14333380957397163, 'feature_fraction': 0.4690000754748763, 'lambda_l1': 0.4937008772491574, 'lambda_l2': 2.2888258490505713, 'learning_rate': 0.08533038187327897, 'max_depth': 3, 'max_leaves': 19, 'min_data_in_leaf': 4, 'objective': 'regression', 'metrics': ['l1', 'l2', 'huber']}
2020-08-30 15:42:51.455 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Rattus_norvegicus', 'Suricata_suricatta']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[88]	valid_0's l1: 0.370175	valid_0's l2: 0.267517	valid_0's huber: 0.123547


2020-08-30 15:42:53.739 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Gorilla_gorilla', 'Cavia_porcellus']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[44]	valid_0's l1: 1.13262	valid_0's l2: 2.19211	valid_0's huber: 0.716867


2020-08-30 15:42:55.457 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Tupaia_belangeri', 'Mesocricetus_auratus']


...

2020-08-30 15:44:16.387 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 2 | VALIDATION_SPECIES: ['Phascolarctos_cinereus', 'Bos_taurus']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[123]	valid_0's l1: 0.494692	valid_0's l2: 0.451215	valid_0's huber: 0.193475


2020-08-30 15:44:20.612 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 3 | VALIDATION_SPECIES: ['Mesocricetus_auratus', 'Homo_sapiens']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[89]	valid_0's l1: 0.477636	valid_0's l2: 0.429008	valid_0's huber: 0.187743


,symbol,repeats,mean_shap,mean_kendall_tau,shap_0,kendall_tau_0,shap_1,kendall_tau_1,shap_2,kendall_tau_2,shap_3,kendall_tau_3,shap_4,kendall_tau_4
ENSG00000087460,GNAS,4.000,19.062,0.333,17.250,0.373,19.250,0.441,18.000,0.218,nan,nan,21.750,0.299
ENSG00000099341,PSMD8,3.000,4.000,0.488,4.750,0.368,2.250,0.578,nan,nan,nan,nan,5.000,0.518
ENSG00000187840,EIF4EBP1,3.000,2.750,0.103,3.500,0.093,nan,nan,1.500,0.249,nan,nan,3.250,-0.033
ENSG00000176731,RBIS,2.000,4.625,-0.442,nan,nan,3.750,-0.456,nan,nan,nan,nan,5.500,-0.428
ENSG00000100220,RTCB,2.000,3.875,-0.453,nan,nan,nan,nan,4.000,-0.533,nan,nan,3.750,-0.373
ENSG00000185838,GNB1L,2.000,3.375,0.361,nan,nan,5.500,0.505,1.250,0.217,nan,nan,nan,nan
ENSG00000134717,BTF3L4,2.000,2.875,0.575,nan,nan,1.750,0.647,nan,nan,nan,nan,4.000,0.503
ENSG00000115204,MPV17,2.000,2.750,-0.611,nan,nan,nan,nan,3.250,-0.656,nan,nan,2.250,-0.566
ENSG00000132423,COQ3,2.000,2.625,0.550,nan,nan,nan,nan,3.500,0.524,nan,nan,1.750,0.576
ENSG00000152380,FAM151B,2.000,1.750,0.363,nan,nan,2.000,0.311,nan,nan,nan,nan,1.500,0.416


### Gestation ###

In [28]:
stage_one_gestation = run_stage_one("gestation_days")
stage_one_gestation.selected

2020-08-30 15:44:31.766 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 0 =====
2020-08-30 15:44:31.767 | INFO     | yspecies.selection:fit:79 - PARAMETERS:
{'bagging_fraction': 0.9550809144864345, 'boosting_type': 'gbdt', 'drop_rate': 0.1520029135951902, 'feature_fraction': 0.39359679175101914, 'lambda_l1': 2.2144413347584315, 'lambda_l2': 2.012506929626787, 'learning_rate': 0.0890025107137548, 'max_depth': 3, 'max_leaves': 18, 'min_data_in_leaf': 5, 'objective': 'regression', 'metrics': ['l1', 'l2', 'huber']}
2020-08-30 15:44:31.789 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Homo_sapiens', 'Pan_paniscus']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[161]	valid_0's l1: 39.8179	valid_0's l2: 4295.93	valid_0's huber: 35.4727


2020-08-30 15:44:35.903 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Capra_hircus', 'Cavia_porcellus']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[145]	valid_0's l1: 15.5697	valid_0's l2: 708.603	valid_0's huber: 13.6444


2020-08-30 15:44:41.359 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Cavia_aperea', 'Microcebus_murinus']


...

2020-08-30 15:47:08.905 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 2 | VALIDATION_SPECIES: ['Capra_hircus', 'Meriones_unguiculatus']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[67]	valid_0's l1: 15.3828	valid_0's l2: 550.402	valid_0's huber: 13.4608


2020-08-30 15:47:11.432 | INFO     | yspecies.selection:fit:82 - SEED: 4 | FOLD: 3 | VALIDATION_SPECIES: ['Pan_paniscus', 'Homo_sapiens']


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[78]	valid_0's l1: 37.5402	valid_0's l2: 3736.82	valid_0's huber: 33.4197


,symbol,repeats,mean_shap,mean_kendall_tau,shap_0,kendall_tau_0,shap_1,kendall_tau_1,shap_2,kendall_tau_2,shap_3,kendall_tau_3,shap_4,kendall_tau_4
ENSG00000006282,SPATA20,5.000,6.500,0.658,5.750,0.668,5.000,0.618,9.750,0.660,5.500,0.670,6.500,0.672
ENSG00000171121,KCNMB3,5.000,5.800,0.629,6.000,0.659,4.250,0.630,6.750,0.609,7.750,0.604,4.250,0.643
ENSG00000172531,PPP1CA,5.000,5.750,-0.775,8.250,-0.828,7.000,-0.769,4.250,-0.751,4.750,-0.732,4.500,-0.794
ENSG00000105672,ETV2,5.000,4.650,0.692,4.750,0.714,4.500,0.697,4.750,0.663,5.000,0.710,4.250,0.676
ENSG00000154328,NEIL2,5.000,2.700,0.600,3.250,0.637,3.250,0.610,2.250,0.595,2.250,0.585,2.500,0.573
ENSG00000010219,DYRK4,4.000,12.938,0.503,12.750,0.701,13.250,0.644,13.250,0.369,nan,nan,12.500,0.297
ENSG00000066923,STAG3,4.000,6.062,0.391,6.500,0.399,5.250,0.357,6.250,0.392,nan,nan,6.250,0.415
ENSG00000188747,NOXA1,4.000,3.625,0.648,2.500,0.658,3.500,0.605,3.750,0.650,4.750,0.679,nan,nan
ENSG00000162959,MEMO1,4.000,3.500,-0.265,6.000,-0.217,2.750,-0.286,nan,nan,3.250,-0.201,2.000,-0.354
ENSG00000163584,RPL22L1,4.000,3.375,-0.553,nan,nan,3.250,-0.485,4.250,-0.623,2.000,-0.624,4.000,-0.480


## Intersect first stages ##

In [1]:
def join_genes():
    return [key for key, value in stage_one]

## Second stage selection ##

In [2]:
join_genes()

NameError: name 'stage_one' is not defined